Gpickle

In [ ]:
import pickle
import torch

path = "/home/tommy/Project/PcodeBERT/outputs/embeddings/19/19adb6449355f995b6119cd1ff97c20a0241e2f3b57ee26b6ad3c6c62d6de8a4.gpickle"

with open(path, 'rb') as f:
    data = pickle.load(f)

print(data.keys())

for node_id, embedding in data['node_embeddings'].items():
    print(f"Node ID: {node_id}")
    print(f"Embedding: {embedding}")

Embedding

In [ ]:
import os
import sys
import pickle
import torch
import numpy as np
from transformers import RobertaForMaskedLM, AutoTokenizer

def load_pretrained_model():
    """載入預訓練的模型和tokenizer"""
    model_path = "/home/tommy/Project/PcodeBERT/outputs/models/pretrain"
    
    print(f"Loading model from: {model_path}")
    
    # 載入tokenizer和model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = RobertaForMaskedLM.from_pretrained(model_path)
    
    # 設定device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print(f"Model loaded successfully on device: {device}")
    return model, tokenizer, device

def get_sentence_embedding(sentence, model, tokenizer, device):
    """對單個sentence生成embedding"""
    # Tokenize
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 生成embedding
    with torch.no_grad():
        outputs = model.roberta(**inputs)
        # 使用[CLS] token的embedding
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    
    return embedding[0]  # 返回一維array

def process_graph_data(graph_path):
    """處理graph資料並生成embeddings"""
    # 載入模型
    model, tokenizer, device = load_pretrained_model()
    
    # 載入graph資料
    print(f"Loading graph data from: {graph_path}")
    with open(graph_path, 'rb') as f:
        graph = pickle.load(f)
    
    print(f"Graph type: {type(graph)}")
    print(f"Number of nodes: {graph.number_of_nodes()}")
    
    # 處理每個節點
    node_embeddings = {}
    sentences = []
    
    for node_id, node_data in graph.nodes(data=True):
        sentence = node_data.get('sentence', '')
        if sentence:
            print(f"Processing node {node_id}: {sentence[:50]}...")
            
            # 生成embedding
            embedding = get_sentence_embedding(sentence, model, tokenizer, device)
            node_embeddings[node_id] = embedding
            sentences.append(sentence)
    
    print(f"Generated embeddings for {len(node_embeddings)} nodes")
    print(f"Embedding dimension: {len(list(node_embeddings.values())[0])}")
    
    return node_embeddings, sentences

def main():
    # 測試資料路徑
    graph_path = "/home/tommy/Project/PcodeBERT/outputs/gpickle/00/00a3a8743be45f6c561c08be96b325c9d6c0a8b619dc67a4e44c5423bcde5532.gpickle"
    
    try:
        # 處理資料
        node_embeddings, sentences = process_graph_data(graph_path)
        
        # 顯示結果
        print("\n=== Embedding Results ===")
        for i, (node_id, embedding) in enumerate(list(node_embeddings.items())[:3]):  # 只顯示前3個
            print(f"Node {node_id}:")
            print(f"  Sentence: {sentences[i][:100]}...")
            print(f"  Embedding shape: {embedding.shape}")
            print(f"  Embedding mean: {np.mean(embedding):.4f}")
            print(f"  Embedding std: {np.std(embedding):.4f}")
            print()
        
        # 儲存結果
        output_path = "/home/tommy/Project/PcodeBERT/outputs/embeddings/graph_embeddings.pkl"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        result = {
            'node_embeddings': node_embeddings,
            'sentences': sentences,
            'embedding_dim': len(list(node_embeddings.values())[0])
        }
        
        with open(output_path, 'wb') as f:
            pickle.dump(result, f)
        
        print(f"Results saved to: {output_path}")
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


In [ ]:
import os
import sys
import pickle
import torch
import numpy as np
from transformers import RobertaForMaskedLM, AutoTokenizer

def load_pretrained_model():
    """載入預訓練的模型和tokenizer"""
    model_path = "/home/tommy/Project/PcodeBERT/outputs/models/pretrain"
    
    print(f"Loading model from: {model_path}")
    
    # 載入tokenizer和model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = RobertaForMaskedLM.from_pretrained(model_path)
    
    # 設定device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print(f"Model loaded successfully on device: {device}")
    return model, tokenizer, device

graph_path = "/home/tommy/Project/PcodeBERT/outputs/gpickle/00/00a3a8743be45f6c561c08be96b325c9d6c0a8b619dc67a4e44c5423bcde5532.gpickle"

with open(graph_path, 'rb') as f:
    data = pickle.load(f)

sample_sentence = data.nodes["0x10002c24L"].get('sentence', '')
print(f"Sample sentence: {sample_sentence}")


model, tokenizer, device = load_pretrained_model()

inputs = tokenizer(sample_sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model.roberta(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
print(f"Embedding shape: {embedding.shape}")

Adapters

In [ ]:
import os
import re
import json
import pickle
import random
import csv

# 錯誤 1: 這個變數沒有被使用，予以刪除 (或保留，如果你未來會用到)
# OPCODE_PAT = re.compile(r"(?:\)\s+|---\s+)([A-Z_]+)") 

OPERAND_PAT = re.compile(r"\(([^ ,]+)\s*,\s*[^,]*,\s*([0-9]+)\)")

def _map_operand_BERT(op_type: str) -> str:
    op_type_l = op_type.lower()
    if op_type_l == 'register':
        return "REG"
    if op_type_l == 'ram':
        return "MEM"
    if op_type_l in {'const', 'constant'}:
        return "CONST"
    if op_type_l == 'unique':
        return "UNIQUE"
    if op_type_l == 'stack':
        return "STACK"
    else:
        return op_type.upper()


def create_instruction_sentence_BERT(instruction):
    opcode = instruction.get("opcode")
    if not opcode:
        return None
        
    tokens = [opcode]
    operation_str = instruction.get("operation", "")
    
    for match in OPERAND_PAT.finditer(operation_str):
        raw_operand_type = match.group(1)
        standardized_token = _map_operand_BERT(raw_operand_type)
        tokens.append(standardized_token)
    
    return tokens


def extract_sentences_from_file_BERT(file_name_data):
    file_name, pcode_dict = file_name_data
    all_functions = {}
    
    try:
        # --- 修正 2 (關鍵)：使用函式名稱 (function_name) 而非位址 (func_addr) ---
        
        # 變數改名 'func_name' -> 'func_addr' (函式位址)，提高可讀性
        for func_addr, func_data in pcode_dict.items():
            if not isinstance(func_data, dict):
                continue
                
            # 獲取真正的函式名稱 (e.g., "ds_is_full")
            real_func_name = func_data.get("function_name")
            if not real_func_name:
                continue # 如果沒有 function_name，則跳過
            
            tokens = []
            for instruction in func_data.get("instructions", []):
                instruction_tokens = create_instruction_sentence_BERT(instruction)
                if not instruction_tokens:
                    continue
                    
                if len(tokens) + len(instruction_tokens) > 512:
                    break
                    
                tokens.extend(instruction_tokens)
            
            if tokens:
                sentence = " ".join(tokens)
                # 使用真正的函式名稱作為 key
                all_functions[real_func_name] = sentence
        # --- 修正 2 結束 ---
            
    except Exception as e:
        # 將 'file_name' 加入錯誤訊息中，方便除錯
        print(f"Error processing file {file_name}: {e}")
    return all_functions


def parse_folder_name(folder_name):
    parts = folder_name.split('_')
    if len(parts) >= 6:
        binary_name = parts[0]
        opt_level = parts[4]
        file_name = parts[5]
        return binary_name, opt_level, file_name
    return "", "", ""


def load_data_from_folder(folder_path, archs):
    arch_data = {arch: {} for arch in archs}
    
    for root, _, files in os.walk(folder_path):
        subfolder = os.path.basename(root)
        arch = next((a for a in archs if a in subfolder), None)
        if not arch:
            continue

        binary_name, opt_level, file_name = parse_folder_name(subfolder)
        
        for fname in files:
            if not fname.endswith(".json"):
                continue
                
            path = os.path.join(root, fname)
            
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                # 'functions' 現在會是 { "func_name": "sentence", ... }
                functions = extract_sentences_from_file_BERT((fname, data))
                
                for func_name, sentence in functions.items():
                    # 'func_name' 現在是 "ds_is_full" 而不是 "0x3bf54L"
                    # 'key' 會是 "which::ds_is_full"，這才能正確跨架構比對
                    key = f"{file_name}::{func_name}"
                    arch_data[arch][key] = (binary_name, opt_level, file_name, func_name, sentence)
                        
            # --- 修正 3：不要 silently pass，至少印出錯誤 ---
            except Exception as e:
                print(f"Error loading or processing file {path}: {e}")
                pass
            # --- 修正 3 結束 ---
                
    return arch_data


def extract_and_save():
    input_folders = ["/home/tommy/Project/PcodeBERT/outputs/binkit_os"]
    output_path = "/home/tommy/Project/PcodeBERT/outputs/alignment_sentences"
    
    source_arch = "x86_64"
    target_archs = ["arm_32", "mips_32"]
    all_archs = [source_arch] + target_archs
    
    arch_datasets = []
    for folder in input_folders:
        arch_datasets.append(load_data_from_folder(folder, all_archs))
    
    for target_arch in target_archs:
        common_keys = None
        for arch_data in arch_datasets:
            keys = set(arch_data[source_arch]) & set(arch_data[target_arch])
            common_keys = keys if common_keys is None else common_keys & keys
        
        # 檢查 common_keys 是否為 None (如果一個資料夾都沒讀到)
        if common_keys is None:
            print(f"No common keys found for {target_arch}. Skipping.")
            continue
            
        common_keys = list(common_keys)
        random.shuffle(common_keys)
        
        n = len(input_folders)
        # 避免 n=0 或 common_keys 為空時出錯
        if n == 0 or not common_keys:
             print(f"No data or folders for {target_arch}. Skipping.")
             continue
             
        chunk_size = len(common_keys) // n
        # 確保 chunk_size 至少為 1 (如果 common_keys < n)
        if chunk_size == 0 and common_keys:
            chunk_size = 1
            
        key_groups = [common_keys[i*chunk_size:(i+1)*chunk_size] for i in range(n)]
        # 處理餘數，將剩餘的 keys 加到最後一個 group
        if common_keys[n*chunk_size:]:
            key_groups[-1].extend(common_keys[n*chunk_size:])
        
        samples = []
        for arch_data, keys in zip(arch_datasets, key_groups):
            for key in keys:
                # 確保 key 真的存在 (雖然 & set 應該保證了，但多一層防護)
                if key in arch_data[source_arch] and key in arch_data[target_arch]:
                    binary_name, opt_level, file_name, func_name, source_func = arch_data[source_arch][key]
                    _, _, _, _, target_func = arch_data[target_arch][key]
                    
                    samples.append((binary_name, opt_level, file_name, func_name, source_func, target_func, 1))
        
        if not samples:
            print(f"No samples generated for {target_arch}. Skipping output.")
            continue
            
        os.makedirs(output_path, exist_ok=True)
        
        pickle_file = os.path.join(output_path, f"train_{source_arch}_{target_arch}_functions.pickle")
        with open(pickle_file, "wb") as f:
            pickle.dump([(source_func, target_func, label) for _, _, _, _, source_func, target_func, label in samples], f)
        
        csv_file = os.path.join(output_path, f"train_{source_arch}_{target_arch}_functions.csv")
        with open(csv_file, "w", newline="", encoding='utf-8') as f: # 增加 encoding='utf-8'
            writer = csv.writer(f)
            writer.writerow(["binary_opt_file_function", f"{source_arch}_function", f"{target_arch}_function", "label"])
            for binary_name, opt_level, file_name, func_name, source_func, target_func, label in samples:
                first_col = f"{binary_name}_{opt_level}_{file_name}_{func_name}"
                writer.writerow([first_col, source_func, target_func, label])

        print(f"Successfully saved {len(samples)} samples for {source_arch} vs {target_arch}")


if __name__ == "__main__":
    extract_and_save()

Adapters deduplicated